# Introducción

## Descripción
Con este notebook de Python se busca introducir la aplicación de técnicas de representación de texto como la bolsa de palabras (BOW por sus siglas en inglés). La representación del texto consiste en transformar el texto (sequencia de símbolos) a una representación más apropiada que permita el uso de distintos algoritmos para así abordar tareas de NLP. Si se quieren utilizar técnicas de machine learning clásico, la etapa de representación del texto implica la extración de características del texto expresadas como un vector del mismo tamaño para todos los documentos con valores númericos.

Las técnicas de repsentación de texto que se verán, son:
* Bolsa de palabras.
* Matriz tf-idf.

En este tutorial también se mostrará como hacer uso de la librería de Python [scikit-learn](https://scikit-learn.org/stable/index.html) para realizar la extracción de características del texto.

**Los objetivos de aprendizaje son**:

1. Explorar técnicas de representación de texto.
2. Hacer uso de librerías para realizar extracción de características del texto.

## Metodología
Este notebook será un tutorial para aprender a instalar y usar **scikit-learn** con el fin de aplicar distintas técnicas de extracción de características del texto. Adicionalmente, se referirá al estudiante a la documentación de la librería y de los métodos vistos para que pueda ampliar la información sobre su uso.

# Instalación de la librería

In [ ]:
# Para instalar localmente, descomente la siguiente línea
# pip install -U scikit-learn

Importación de los módulos a usar en el tutorial

In [3]:
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Representación de bolsa de palabras (BOW)

La librería scikit-learn nos permite realizar de manera sencilla la transformación del texto a una representación de bolsa de palabras. Esta representación consiste en representar cada documento como un conjunto que registra la ocurrencia de cada una de las palabras del vocabulario en el documento.

Para realizar esta transformación, harémos uso de la clase [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), la cual realiza esta vectorización con una interfaz sencilla, pero fácilmente ajustable a nuestras necesidades específicas.

El caso más simple de uso de CountVectorizer, se compone de dos pasos. El primero, es la inicialización de la clase, y el segundo la transformación del corpus de documentos en los vectores de ocurrencia.

In [4]:
count_vect = CountVectorizer()
X = count_vect.fit_transform(['This is is the test 1', 'This is the the the test 2', 'This is the test 3'])

print('Vocabulario:', count_vect.get_feature_names_out())

print('Dimensiones de la matriz X:', X.shape)

print('Contenido de la matriz X:\n', X.toarray())

print('Cantidad de documentos:', X.shape[0])
print('Cantidad de palabras:', X.shape[1])
print('Cantidad de ocurrencias:', X.sum())

# Ocurrencias por documento
print('Tokens en el documento', X.sum(axis=1))

# Ocurrencias por palabra
print('Ocurrencias por token', X.sum(axis=0))

Vocabulario: ['is' 'test' 'the' 'this']
Dimensiones de la matriz X: (3, 4)
Contenido de la matriz X:
 [[2 1 1 1]
 [1 1 3 1]
 [1 1 1 1]]
Cantidad de documentos: 3
Cantidad de palabras: 4
Cantidad de ocurrencias: 15
Tokens en el documento [[5]
 [6]
 [4]]
Ocurrencias por token [[4 3 5 3]]


Con los valores por defecto CountVectorizer realiza un procesamiento del texto y una transformación que se compone de los siguientes pasos:

1. Se realiza una tokenización del texto en donde solo se conservan las secuencias de caracteres de 2 o más caracteres alfanuméricos.
2. Se realiza una normalización de los tokens en donde se transforman a minúsculas.
3. Se contruye el vocabulario compuesto de todos los tokens presentes en el corpus.
4. Se construye la matriz de ocurrencia de términos con los documentos como filas, y el índice de los tokens del vocabulario como columnas.

Nota: La matriz resultado es un objeto tipo `scipy.sparse` para hacer un uso eficiente de la memoria, ya que al representar cada documento con un vector de tamaño del vocabulario, estos son muy grandes y están compuestos principalmente por ceros (dispersos).

Para explorar la representación vectorial de cada documento, puede ver el vector mediante el índice del documento en el corpus. Por ejemplo, a continuación se muestra el vector correspondiente al primer documento del corpus:

In [ ]:
X[0].toarray()

array([[2, 1, 1, 1]])

Reflexione un momento en la razón por la cual este vector tiene un "2" en una de sus dimensiones. ¿Que palabra se repite dos veces en este documento?, ¿Volviendo al vector del vocabulario en que posición/dimensión se encuntra dicha palabra?, ¿Coincide con la dimensión donde apararece el 2 en el documento?

También puede observar facilmente los términos del vocabulario presentes en el primer documento usando las posiciones diferentes de zero del vector que representa el documento:

In [ ]:
count_vect.get_feature_names_out()[X[0].nonzero()[1]]

array(['this', 'is', 'the', 'test'], dtype=object)

En este ejercicio el vocabulario es extremandamente pequeño, solo 4 tokens/palabras, y es por eso que los vectores que representan a los documentos no son dispersos. En un escenario real con vocabularios compuestos por miles de palabras distintas esto no va a ocurrir, y la mayor parte de las posiciones resultantes del vector que representa a un documento van a ser cero.

## Controlar el procesamiento de texto

En algunas situaciones, puede ser útil el poder configurar la etapa de procesamiento de texto de acuerdo con nuestras necesidades. Para lograrlo, podemos pasar distintos parámetros en la incialización de CountVectorizar que nos permiten ajustar la tokenización del texto, y las técnicas de normalización que queremos aplicar.

### Transformación a minúsculas

La transformación a minúsculas se hace por defecto, pero se puede desactivar mediante el argumento **lowercase**.

In [ ]:
count_vect = CountVectorizer(lowercase=False)
sample = count_vect.fit_transform(['Hola, ¿cómo estás?'])
count_vect.get_feature_names_out()

array(['Hola', 'cómo', 'estás'], dtype=object)

### Remover acentos

Si se quieren eliminar la acentuación de los tokens, podemos pasar como argumento a **strip_accents** el valor de `unicode`.

In [ ]:
count_vect = CountVectorizer(strip_accents='unicode')
sample = count_vect.fit_transform(['Hola, ¿cómo estás?'])
count_vect.get_feature_names_out()

array(['como', 'estas', 'hola'], dtype=object)

### Remover palabras de parada
Para remover las palabras de parada del corpus, podemos pasar un listado con las palabras que queremos ignorar en el argumento **stop_words**.

In [ ]:
stops = nltk.corpus.stopwords.words('spanish')

count_vect = CountVectorizer(stop_words=stops)
sample = count_vect.fit_transform(['Hola, ¿cómo estás?'])
count_vect.get_feature_names_out()

array(['cómo', 'hola'], dtype=object)

### Modificar la tokenización

Si queremos definir los tokens según criterios diferentes a los usados por defecto, podemos pasar una función en donde definamos la tokenización o podemos pasar un patrón de expresiones regulares con nuestras reglas de tokenización.

Vea en el ejemplo siguiente como los números que contienen comas o puntos se covierten en varios tokens, solo se convierte en un token las palabras unidas por un guíon, las siglas se ignoran, al igual que los signos de puntuación:

In [ ]:
count_vect = CountVectorizer()
sample = count_vect.fit_transform(['Hola, ¿cómo estás? 1,234.56, 1,233,123 test-test U.S.A. mi_nombre'])
count_vect.get_feature_names_out()[sample[0].nonzero()[1]]

array(['hola', 'cómo', 'estás', '234', '56', '233', '123', 'test',
       'mi_nombre'], dtype=object)

A continuación definimos nuestra propia regla de tokenización por medio de una expresión regular:

In [ ]:
regex = r"""
(?x) # Permite agregar comentarios y espacios en la expresión
(?:(?:\d+,?)+(?:\.\d+)?) # números con comas y puntos como decimales
| (?:[^\W\d_]\.)+ # Palabras con puntos (abreviaciones)
| \w+(?:-\w+)* # Palabras, acepta guiones intermedios (test-test)
| ['\"¡¿.,!?]+ # Acepta signos de puntuación
"""
count_vect = CountVectorizer(token_pattern=regex)
sample = count_vect.fit_transform(['Hola, ¿cómo estás? 1,234.56, 1,233,123 test-test U.S.A. mi_nombre'])
count_vect.get_feature_names_out()[sample[0].nonzero()[1]]

array(['hola', ',', '¿', 'cómo', 'estás', '?', '1,234.56', '1,233,123',
       'test-test', 'u.s.a.', 'mi_nombre'], dtype=object)

## Controlando la creación del vocabulario

Para crear una representación de bolsa de palabras, primero se realiza la creación del vocabulario. Por defecto, el vocabulario contiene todos los tokens presentes en el corpus. Sin embargo, **scikit-learn** nos permite controlar como se crea el vocabulario, definir que palabras ignorar, o definir el vocabulario a utilizar.

En este ejemplo, vea como podemos predefinir el vocabulario con el cual se va a crear la representación de bolsa de palabras:

In [ ]:
vocab = { # Los índices (valores) deben ser únicos e ir de 0 a N-1, donde N es la cantidad de tokens en el vocabulario
    'hola': 0,
    'como': 1,
    'estas': 2,
    'test': 3,
}

count_vect = CountVectorizer(vocabulary=vocab)
sample = count_vect.fit_transform(['Hola, ¿cómo estás?', 'este es el test test 2', 'test test test 3'])

print('Vocabulario creado:', count_vect.get_feature_names_out())
sample.toarray()

Vocabulario creado: ['hola' 'como' 'estas' 'test']


array([[1, 0, 0, 0],
       [0, 0, 0, 2],
       [0, 0, 0, 3]])

Nótese que las palabras pasadas en el vocabulario deben coincidir exactamente con las palabras tokenizadas y normalizadas de los documentos. Por ejemplo, en este caso las palabras del vocabulario "como" y "estas" tienen cero ocurrencias en los documentos debido a que en el vocabulario hay ausencia de las tildes.

## Limite el tamaño del vocabulario

Para limitar el tamaño del vocabulario, mediante el argumento **max_features** puede definir la cantidad máxima de palabras presentes en el vocabulario, así el vocabulario solo conservará los tokens con mayor ocurrencia en el corpus.

In [ ]:
count_vect = CountVectorizer(max_features=3)
sample = count_vect.fit_transform(['Hola, ¿cómo estás?', 'este es el test test 2', 'test test test 3'])

print('Vocabulario creado:', count_vect.get_feature_names_out())
sample.toarray()

Vocabulario creado: ['cómo' 'el' 'test']


array([[1, 0, 0],
       [0, 1, 2],
       [0, 0, 3]])

Adicionalmente, puede ignorar del vocabulario las palabras o tokens que están presentes en casi todos los documentos del corpus. Esto sería el equivalente a sleccionar palabras de parada de acuerdo a su corpus, es decir, palabras o tokens que al estar presentes en casi todos los documentos poseen un bajo poder discriminativo, al no conllevar información relevante para diferenciar los documentos.

Para lograr esto puede definir el argumento **max_df**. En el vocabulario solo se agregarán los tokens con una frecuencia menor a max_df en el corpus, donde max_df puede ser una proporción del total de documentos o un valor absoluto.

In [ ]:
count_vect = CountVectorizer(max_df=0.7)
sample = count_vect.fit_transform(['Hola, ¿cómo estás?', 'este es el test test 2', 'test test test 3', 'Este es otro test'])

print('Vocabulario creado:', count_vect.get_feature_names_out())
sample.toarray()

Vocabulario creado: ['cómo' 'el' 'es' 'este' 'estás' 'hola' 'otro']


array([[1, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 1]])

La palabra test que se repite en $3/4=0.75$ de los documentos, por lo tanto no se agrega al vocabulario.

También se puede ignorar palabras con una frecuencia por documento menor a un valor determinado, esta técnica puede ser útil para eliminar palabras poco representativas del corpus que podrían ser vistas como outliers o errores.

In [ ]:
count_vect = CountVectorizer(min_df=0.3)
sample = count_vect.fit_transform(['Hola, ¿cómo estás?', 'este es el test test 2', 'test test test 3', 'Este es otro test'])

print('Vocabulario creado:', count_vect.get_feature_names_out())
sample.toarray()

Vocabulario creado: ['es' 'este' 'test']


array([[0, 0, 0],
       [1, 1, 2],
       [0, 0, 3],
       [1, 1, 1]])

Para este caso, las palabras con presencia en solo uno de los 4 documentos se ignoran en la ceación del vocabulario.

# Matriz tf-idf

Esta representación de texto es una variante de la representación en bolsa de palabras, en donde en vez de registrar la ocurrencia se registra el score tf-idf, el cual es una función que pondera la ocurrencia del termino con el poder discriminativo del mismo.

En **scikit-learn** podemos aplicar esta representación de manera muy similar a CountVectorizer con la clase [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer).

In [ ]:
corpus = ['Hola, ¿cómo estás?', 'este es el test test 2', 'test test test 3', 'Este es otro test']

tfidf_vect = TfidfVectorizer()
sample = tfidf_vect.fit_transform(corpus)

print('Vocabulario creado:', count_vect.get_feature_names_out())
sample.toarray()

Vocabulario creado: ['cómo' 'el' 'es' 'este' 'estás' 'hola' 'otro' 'test']


array([[0.57735027, 0.        , 0.        , 0.        , 0.57735027,
        0.57735027, 0.        , 0.        ],
       [0.        , 0.50814302, 0.40062579, 0.40062579, 0.        ,
        0.        , 0.        , 0.64868222],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.4842629 , 0.4842629 , 0.        ,
        0.        , 0.61422608, 0.39205255]])

Los scores tf-idf de **scikit-learn** se calculan de la siguiente manera:

$\text{tf-idf} = \text{tf} * \text{idf}$, en donde **tf** es la ocurrencia del término en el corpus.

El **idf** se obtiene así:

$\text{idf} =  \ln({1 + n\over 1 + df}) + 1$, donde **n** es el número de documentos en el corpus, y **df** es el número de documentos en el corpus que contienen el término.

Finalmente, se normaliza cada vector de documento, de modo que cada vector tenga una norma de una unidad.

In [ ]:
counts = CountVectorizer().fit_transform(corpus)
tf = counts.toarray()
df = np.where(tf > 0, 1, 0)

idf = np.log((1 + df.shape[0]) / (1 + df.sum(axis=0))) + 1

tfidf = tf * idf
tfidf /= np.linalg.norm(tfidf, axis=1, keepdims=True)
tfidf

array([[0.57735027, 0.        , 0.        , 0.        , 0.57735027,
        0.57735027, 0.        , 0.        ],
       [0.        , 0.50814302, 0.40062579, 0.40062579, 0.        ,
        0.        , 0.        , 0.64868222],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.4842629 , 0.4842629 , 0.        ,
        0.        , 0.61422608, 0.39205255]])